# Retrieval State Machine

This is a single-purpose state machine. It is characterized by a single line connecting all the states.

## State 0. Init

The starting point is to create the string representation of the following mermaid diagram as follows:

1. Add **CRAFT_TOOL_PROMPT**
2. Add **TOOL_CALL** transition to **RETRIEVAL: next / on_RETRIEVAL**.
3. Add **retrieval** to tools_dict.



In [1]:
state_diagram = """stateDiagram-v2
    INIT --> CRAFT_TOOL_PROMPT: next / on_CRAFT_PROMPT
        
    CRAFT_TOOL_PROMPT --> TOOL_CALL: next / on_TOOL_CALL        
        TOOL_CALL --> RETRIEVAL: next / on_RETRIEVAL
        RETRIEVAL --> GENERATE: next / on_GENERATE

    GENERATE --> PROCESS: next / on_PROCESS
        PROCESS --> END : next
    """

```mermaid
stateDiagram-v2
    INIT --> CRAFT_TOOL_PROMPT: next / on_CRAFT_PROMPT
        
    CRAFT_TOOL_PROMPT --> TOOL_CALL: next / on_TOOL_CALL        
        TOOL_CALL --> RETRIEVAL: next / on_RETRIEVAL
        RETRIEVAL --> GENERATE: next / on_GENERATE

    GENERATE --> PROCESS: next / on_PROCESS
        PROCESS --> END : next
```

In [2]:
# ARRANGE
import os
os.environ["LOG_LEVEL"]="WARNING"
#state_diagram_path = os.path.dirname(os.path.abspath(__name__)) + "/simple_agent_state_diagram.md"

from gai.ttt.client.ttt_client import TTTClient
ttt = TTTClient({
    "type": "ttt",
    "url": "http://localhost:12031/gen/v1/chat/completions"
})

from gai.rag.client.rag_client_async import RagClientAsync
rag = RagClientAsync({
    "type": "rag",
    "url": "http://localhost:12036/gen/v1/rag",
    "ws_url": "ws://localhost:12036/gen/v1/rag/index-file/ws"
})

from gai.persona.profile.pydantic.AgentPydantic import AgentPydantic
agent_data = AgentPydantic(
    Id="",
    Name="Alfred",
    AgentDescription="Hello, I am Alfred, the best large language model in the world, and I am here to assist you in any way possible. As a highly advanced AI assistant, I possess the ability to perform general-purpose tasks powered by <span ${style}>GPT-4</span>. With my vast knowledge base and powerful processing capabilities, I am able to provide you with the most relevant and helpful information available. Whether you need answers to complex questions, recommendations for products or services, or assistance with decision making, I am here to help. So, how may I be of service to you today?",
    AgentTraits="Wise,Serious,Meticulous",
    ImageUrl="",
    ThumbnailUrl=""
    )
doc_titles=["MICHELIN Guide Singapore 2016: Where to Find the Best Chicken Rice in Singapore"]
doc_keywords=["Singapore","Chicken Rice","Food"]


In [3]:
tools_config={
    "retrieval": {
        "tool_prompt":"👩‍🔬, use only the information provided to you by the user to answer the user''s question. \n            If the information is insufficient for 👩‍🔬 to derive an answer, just say ''I cannot find relevant information in my document store to answer the question correctly.'' \n            👩‍🔬 is to provide an in-depth analysis to the question based only on the information provided by the user and nothing more.\n            👩‍🔬 will give a real-life example to support illustrating your point and contrasting it with a counter-example. \n            👩‍🔬 will also proofread and edit the content before responding. \n            👩‍🔬 will provide your own reasoned subjective perspective, noting where your view differs from or expands on the contents.\n            Rules:\n                - Consolidate the materials provided by the user and then organise them point by point.\n                - Provide as much details as you can extract from the materials provided by the user.\n                - Begin your report by saying `According to my document store,...`\n                - Always provide your answers in point form.",
        "schema": {
            "type": "function",
            "function": {
                "name": "retrieval",
                "description": f"""
                    The `retrieval` function is a powerful tool that allows the AI to access articles outside of its knowledge domain from external sources. 
                    The external articles are stored in an archive and organised by <titles>:\n{{ titles: [{doc_titles}] }}
                    and <keywords>:
                    {{ keywords: [{doc_keywords}] }}
                    **IMPORTANT**: Use this tool when any of the <titles> or <keywords> may be relevant to user's question.
                    The \'search_query\' parameter should be crafted in a way that it returns the most precise result based on the conversation context.
                """,
                "arguments": {
                    "type": "object",
                    "properties": {
                        "search_query": {
                            "type": "string",
                            "description": """The most effective search query for semantic search that will return the most precise result."""
                        }
                    },
                    "required": ["search_query"]
                }
            }                    
        }
    },
}

This is a single action state machine

In [7]:
from gai.persona.fsm.AgentStateMachine import AgentStateMachine
agent = AgentStateMachine(
    ttt=ttt,
    rag=rag,
    agent_data=agent_data,
    collection_name="demo",
    dialogue_messages=[],
    user_message="Which chicken rice restaurant also sells char siew?",
    tools_config=tools_config,
    state_diagram=state_diagram,
    tool_name="retrieval"
    ).Init()


Machine validation successful.


## State 2. CRAFT_TOOL_PROMPT

In [8]:
print("before:"+agent.state)
agent.next()
print("content:",agent.content)
print("after:"+agent.state)


before:INIT
content: {'system_message': ''}
after:CRAFT_TOOL_PROMPT


## State 3. TOOL_CALL

In [9]:
print("before:"+agent.state)
agent.next()
print("content:",agent.content)
print("after:"+agent.state)

before:CRAFT_TOOL_PROMPT
content: {"type": "function", "name": "retrieval", "arguments": "{\"search_query\": \"chicken rice restaurant char siew\"}"}
after:TOOL_CALL


## State 4. RETRIEVAL

In [10]:
print("before:"+agent.state)
agent.next()
print("content:",agent.content)
print("after:"+agent.state)

before:TOOL_CALL
content: ["well as salt-baked chicken. With the diversity presented in a single dish, Singapore's chicken rice definitely draws in curious diners to uncover more of the Lion City's chicken rice culture. Being equally fascinated with Singapore's chicken rice, the MICHELIN Guide inspectors share with us where to get the best ones in Singapore, so read on below! Boon Tong Kee (Balestier Road) 399, 401, 403 Balestier Road, 329801 Singapore $", "· Cantonese Roast Meats A household brand that's popular both in Singapore and overseas, Hawker Chan Soya Sauce Chicken Rice & Noodle 's soya sauce chicken is well-known far and wide. The chicken is cooked in the chef's secret sauce for several hours, and it yields tender and flavourful meat. First-time diners are recommended to try the signature soya sauce chicken rice to experience its charming qualities. More Details Xing Yun Hainanese Boneless Chicken Rice Yuhua Market & Hawker Centre, #01-202, Blk 347, Jurong East Avenue 1, Jur

## State 5. GENERATE

In [11]:
print("before:"+agent.state)
agent.next()
print("after:",agent.state)
for chunk in agent.streamer:
    print(chunk,end="",flush=True)

before:RETRIEVAL
after: GENERATE
 According to my document store, Tian Tian Hainanese Chicken Rice also sells char siew. This information is not explicitly stated in the context provided, but it can be inferred from the fact that Tian Tian Hainanese Chicken Rice is a Cantonese roast meat restaurant, and Cantonese cuisine often includes char siew (bbq pork) in their menu

## State 6. PROCESS

In [12]:
print("before:"+agent.state)
agent.next()
print("content:",agent.content)
print("after:",agent.state)

before:GENERATE
content:  According to my document store, Tian Tian Hainanese Chicken Rice also sells char siew. This information is not explicitly stated in the context provided, but it can be inferred from the fact that Tian Tian Hainanese Chicken Rice is a Cantonese roast meat restaurant, and Cantonese cuisine often includes char siew (bbq pork) in their menu
after: PROCESS
